In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [7]:
bikes["total_rentals"] = bikes["casual"] + bikes["registered"]

bikes["dteday"] = pd.to_datetime(bikes["dteday"])
bikes["day"] = bikes["dteday"].dt.day
bikes["month"] = bikes["dteday"].dt.month
bikes["year"] = bikes["dteday"].dt.year

features = [
    "hr", "temp_c", "feels_like_c", "hum", "windspeed",
    "weathersit", "season", "holiday", "workingday",
    "day", "month", "year"
]

X = bikes[features]
y = bikes["total_rentals"]


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike_predictions.csv", index=False, header=['total_rentals'])
print("✅ CSV export complete!")

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 122390.6719 - mae: 236.8696 - val_loss: 71144.5547 - val_mae: 193.2275
Epoch 2/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 71047.5703 - mae: 192.5739 - val_loss: 69995.4141 - val_mae: 191.8652
Epoch 3/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 69283.8594 - mae: 190.0084 - val_loss: 62614.9883 - val_mae: 175.9333
Epoch 4/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 59024.9883 - mae: 170.8264 - val_loss: 51898.6953 - val_mae: 161.1130
Epoch 5/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 50925.7539 - mae: 158.9416 - val_loss: 47681.7148 - val_mae: 153.0690
Epoch 6/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 46480.5781 - mae: 149.7854 - val_loss: 43608.6328 - val_mae: 142.9884
Epoch 7/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 43237.4883 - mae: 141.7621 - val_loss: 40259.3984 - val_mae: 138.2474
Epoch 8/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 39224.8086 - mae: 133.4

# Task
Improve the model's performance in the notebook "/content/module04_biking_grading.ipynb" by iterating on the model architecture, hyperparameters, feature engineering, and regularization techniques, evaluating the results using the provided metrics (Within 5%, Within 10%, Within 20%, R^2, RMSE, MAE, MedAE) after each change, and summarizing the changes and results.

## Revisar la arquitectura del modelo

### Subtask:
Experimentar con un número diferente de capas densas y neuronas en cada capa.


**Reasoning**:
Modify the model architecture with a different number of dense layers and neurons, keeping the input shape and the output layer the same, and then train and evaluate the model.



In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike_predictions.csv", index=False, header=['total_rentals'])
print("✅ CSV export complete!")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 97815.8438 - mae: 216.9100 - val_loss: 52284.4688 - val_mae: 158.3487
Epoch 2/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 47973.1328 - mae: 146.3225 - val_loss: 38673.2812 - val_mae: 129.9340
Epoch 3/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 36106.6602 - mae: 122.3561 - val_loss: 29309.1406 - val_mae: 111.2327
Epoch 4/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 27531.2617 - mae: 105.3005 - val_loss: 21451.1348 - val_mae: 95.9107
Epoch 5/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 19624.0020 - mae: 89.5857 - val_loss: 15647.6338 - val_mae: 80.1259
Epoch 6/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 15214.5195 - mae: 79.4641 - val_loss: 13894.1035 - val_mae: 78.3078
Epoch 7/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 12633.9961 - mae: 72.9016 - val_loss: 13138.8828 - val_mae: 74.7714
Epoch 8/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 11051.2803 - mae: 67